پروژه ی تشخیص سرطان پوستی !!!ایجاد یک مدل شبکه  عصبی کانولوشنی که تصاویر حاوی بیماری های پوستی را دریافت میکند و تشخیص میدهد که خوشخیم است یا بد خیم؟ این کد مدلسازی اولیه هست و به مرور کد رو بهبود میبخشم و یک رابطه کاربری جذاب واسش درست میکنم .

Skin cancer diagnosis project!!! Creating a convolutional neural network model that receives images containing skin diseases and determines whether it is benign or malignant? This is the initial modeling code and I will gradually improve the code and create an attractive user interface for it.

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Input
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
benign_dir = r"C:\Users\number_one\Desktop\prog\poost\train_cancer\benign"
malignant_dir = r"C:\Users\number_one\Desktop\prog\poost\train_cancer\malignant"

In [3]:
images = []
labels = []

In [4]:
for label in  ['benign', 'malignant']:
    folder = benign_dir if label =='benign' else malignant_dir
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder , img_file)
        img = Image.open(img_path)
        img = img.resize((150,150))
        img = np.array(img)
        images.append(img)
        labels.append(0 if label == 'benign' else 1)

In [5]:
images = np.array(images)
labels = np.array(labels)

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [8]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [9]:
model = Sequential()
model.add(Input(shape=(150, 150, 3)))  
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))  
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=8),
                    validation_data=(X_val, y_val),
                    epochs=20)  

Epoch 1/20


c:\Users\number_one\Desktop\prog\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 198ms/step - accuracy: 0.7601 - loss: 0.8531 - val_accuracy: 0.8750 - val_loss: 0.3798
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.9068 - loss: 0.3576 - val_accuracy: 0.8750 - val_loss: 0.6668
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.8690 - loss: 0.4152 - val_accuracy: 0.8750 - val_loss: 0.5707
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.8976 - loss: 0.3043 - val_accuracy: 0.8750 - val_loss: 0.5794
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.9102 - loss: 0.2455 - val_accuracy: 0.8750 - val_loss: 0.5133
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.8973 - loss: 0.2397 - val_accuracy: 0.8750 - val_loss: 0.4424
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.9424 - loss: 0.1769 - val_accuracy: 0.6000 - val_loss: 0.5990
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.9465 - loss: 0.2173 - val_accuracy: 0.8750 - val_

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9571 - loss: 0.1654
Test Accuracy: 95.12%


In [13]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, np.round(predictions)))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.95      1.00      0.97        36

    accuracy                           0.95        41
   macro avg       0.97      0.80      0.86        41
weighted avg       0.95      0.95      0.95        41



In [14]:
model.save('skin_cancer.h5')